In [1]:
import networkx as nx
import os
import pandas as pd
import numpy as np

In [2]:
processed_network = os.path.join(os.getenv("DATA_DIR"),"processed_network")

In [3]:
os.listdir(processed_network)

['taxon_28_31_01_sample30_nodes.csv.gz',
 'taxon_28_31_01_edges_doo.csv.gz',
 'taxon_28_31_01_nodes_doo.csv.gz',
 'taxon_28_31_01_sample30_edges.csv.gz',
 'for_networkx_tutorial_nodes.csv.gz',
 'for_networkx_tutorial_edges.csv.gz']

In [4]:
nodefile = os.path.join(processed_network, "taxon_28_31_01_sample30_nodes.csv.gz")
edgefile = os.path.join(processed_network, "taxon_28_31_01_sample30_edges.csv.gz")

### Load nodes

In [5]:
nodes = pd.read_csv(nodefile, compression="gzip", sep="\t")

In [6]:
nodes.head()

,Node,Node_id,Node_Taxon
0,date of transfer,1038061,"('other',)"
1,/,13,"('other',)"
2,/ /apply-universal-credit,1085030,"('other',)"
3,/ accessibility,918380,"('other',)"
4,/ aeoi,564171,"('other',)"


In [7]:
nodes.shape

(1179923, 3)

In [8]:
nodes = nodes[~nodes.Node.str.contains(" ")]

### Load edges

In [9]:
edges = pd.read_csv(edgefile, compression="gzip", sep="\t")

In [10]:
edges.head()

,Source_node,Source_id,Destination_node,Destination_id,Weight,Source_Taxon,Destination_Taxon
0,/government/publications/guidance-for-dependan...,0,/visa-fees,1,23,"('d612c61e-22f4-4922-8bb2-b04b9202126e',)","('29480b00-dc4d-49a0-b48c-25dda8569325',)"
1,/visa-fees,1,/find-a-visa-application-centre,2,427,"('29480b00-dc4d-49a0-b48c-25dda8569325',)","('d3df7728-db52-411f-afd2-44dbd09f553b',)"
2,/find-a-visa-application-centre,2,/entering-staying-uk/family-visas,3,1,"('d3df7728-db52-411f-afd2-44dbd09f553b',)","('d612c61e-22f4-4922-8bb2-b04b9202126e',)"
3,/entering-staying-uk/family-visas,3,/uk-family-visa,4,148,"('d612c61e-22f4-4922-8bb2-b04b9202126e',)","('d612c61e-22f4-4922-8bb2-b04b9202126e',)"
4,/uk-family-visa,4,/uk-family-visa/partner-spouse,5,10858,"('d612c61e-22f4-4922-8bb2-b04b9202126e',)","('d612c61e-22f4-4922-8bb2-b04b9202126e',)"


#### Filter edge list: remove self-loops, remove edges to/from `/search`

In [11]:
edges[edges.Destination_node.str.contains("/search?")].shape, edges[edges.Destination_node.str.
                                                                    contains("/search?")].Weight.sum()

((758634, 7), 1718465)

In [20]:
edges = edges[(~edges.Source_node.str.contains(" ")) & (~edges.Destination_node.str.contains(" "))]
edges = edges[edges.Source_node!=edges.Destination_node]
edges = edges[(~edges.Source_node.str.contains("/search?")) & (~edges.Destination_node.str.contains("/search?"))]

In [21]:
edges.shape

(2244101, 7)

In [22]:
edges[(edges.Source_node.str.contains("/search?"))]

,Source_node,Source_id,Destination_node,Destination_id,Weight,Source_Taxon,Destination_Taxon


In [23]:
edges.sort_values(by="Weight", ascending=False)[["Source_node","Destination_node","Weight"]].head()

,Source_node,Destination_node,Weight
276,/government/organisations/companies-house,/get-information-about-a-company,6975383
20,/log-in-file-self-assessment-tax-return,/log-in-file-self-assessment-tax-return/sign-i...,3627670
225,/government/organisations/hm-revenue-customs,/log-in-register-hmrc-online-services,3018949
241,/self-assessment-tax-returns,/log-in-file-self-assessment-tax-return,1935164
2478,/universal-credit,/sign-in-universal-credit,1801480


In [24]:
edges.shape, edges[edges.Weight >= 3].shape

((2244101, 7), (433644, 7))

## Initialize functional graph

In [25]:
graph = nx.DiGraph()

In [26]:
# nodes_list = [[tup.Node,{"id":tup.Node_id, "taxon":tup.Node_Taxon}] for tup in nodes.itertuples()]
# print(nodes_list[0:10])
# graph.add_nodes_from(nodes_list)

In [27]:
edge_list = [(tup.Source_node, tup.Destination_node, {"weight":tup.Weight}) 
             for tup in edges.itertuples() if tup.Weight >= 3]

In [28]:
graph.add_edges_from(edge_list)

In [29]:
nx.info(graph)

'Name: \nType: DiGraph\nNumber of nodes: 137160\nNumber of edges: 433644\nAverage in degree:   3.1616\nAverage out degree:   3.1616'

In [30]:
nx.is_connected(graph.to_undirected())

False

In [31]:
len(list(nx.connected_components(graph.to_undirected())))

4207

In [32]:
for i,cc in enumerate(nx.connected_components(graph.to_undirected())):
    if len(cc) > 10000:
        print(i,len(cc))

0 126424


In [33]:
largest_cc = max(nx.connected_components(graph.to_undirected()), key=len)
list(largest_cc)[0:10]

['/hmrc-internal-manuals/business-income-manual/bim39520',
 '/check-if-you-need-tax-return/y/no/between-50-000-and-100-000/no-i-do-not-have-children-or-did-not-claim/no/no/none-of-these/no',
 '/calculate-your-holiday-entitlement/y/casual-or-irregular-hours/28.5',
 '/government/news/government-announces-new-housing-measures',
 '/towing-rules/y/medium-sized-vehicle/no/no/before-jan-1997',
 '/calculate-your-holiday-entitlement/y/hours-worked-per-week/starting/2018-05-09/2018-04-01',
 '/hmrc-internal-manuals/pensions-tax-manual/ptm145100',
 '/state-pension-age/y/age/1959-06-12/female',
 '/hmrc-internal-manuals/international-exchange-of-information/ieim401200',
 '/government/publications/procurement-policy-note-10-12-the-public-services-social-value-act-2012']

In [34]:
subgraph = graph.subgraph(largest_cc)
nx.info(subgraph)

'Name: \nType: DiGraph\nNumber of nodes: 126424\nNumber of edges: 425389\nAverage in degree:   3.3648\nAverage out degree:   3.3648'

    'Name: \nType: DiGraph\nNumber of nodes: 146748\nNumber of edges: 514181\nAverage in degree:   3.5038\nAverage out degree:   3.5038'

In [35]:
attrs = {"node_id":{},"node_taxon":{}}
for tup in nodes.itertuples():
    if tup.Node in subgraph.nodes():
        attrs["node_id"][tup.Node] = tup.Node_id
        attrs["node_taxon"][tup.Node] = tup.Node_Taxon

In [36]:
list(attrs["node_id"].items())[0:10]

[('/', 13),
 ('/-self-assessment-tax-bill', 375786),
 ('/-tax', 229776),
 ('/-tax-debit-credit-card', 111642),
 ('/1619-bursary-fund', 6687),
 ('/1619-bursary-fund/eligibility', 6689),
 ('/1619-bursary-fund/further-information', 45809),
 ('/1619-bursary-fund/how-to-claim', 6690),
 ('/1619-bursary-fund/print', 178760),
 ('/1619-bursary-fund/what-youll-get', 6688)]

In [37]:
nx.set_node_attributes(graph, attrs["node_id"], 'nid')
nx.set_node_attributes(graph, attrs["node_taxon"], 'taxon')

In [38]:
for node,data in list(subgraph.nodes(data=True))[0:2]:
    print(node,data.keys())

/government/publications/guidance-for-dependants-of-uk-visa-applicants-tiers-1-2-4-5 dict_keys(['nid', 'taxon'])
/visa-fees dict_keys(['nid', 'taxon'])


### Compute shortest path

In [39]:
short_paths = nx.shortest_path(subgraph, source="/")

In [42]:
len(short_paths)

121409

In [41]:
count = 0
for key,value in short_paths.items():
    if len(value) > 15:
        count+=1
count

394

In [ ]:
%timeit nx.average_shortest_path_length(subgraph)

### Subtrees from homepage

In [ ]:
home_edges = list(nx.dfs_edges(subgraph, source='/'))

In [ ]:
home_graph = subgraph.edge_subgraph(home_edges).copy()  